# Sitemap Assay

The start of a simple notebook that could hosted for peeopl to test out their sitemaps (and robots.txt) files with.

References:
* [AdvTools](https://advertools.readthedocs.io/en/master/advertools.sitemaps.html)
* [Sitemap viz](https://www.ayima.com/us/insights/analytics-and-cro/how-to-visualize-an-xml-sitemap-using-python.html)


<a href="https://githubtocolab.com/gleanerio/archetype/blob/master/networks/commons/sitemap_assay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.png" alt="Open in Colab"/></a>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/gleanerio/archetype/HEAD?labpath=networks/commons/sitemap_assay.ipynb)



## Notes

has json-ld :  https://researchdata.edu.au/imos-soop-underway-sep-2017/970828
Validation:  https://validator.schema.org/#url=https%3A%2F%2Fresearchdata.edu.au%2Fimos-soop-underway-sep-2017%2F970828  


doesn't:  https://researchdata.edu.au/heupel-michelle/1709766

Validation   https://validator.schema.org/#url=https%3A%2F%2Fresearchdata.edu.au%2Fheupel-michelle%2F1709766


Context as map:  https://www.w3.org/TR/json-ld/#context-definitions





In [3]:
!pip -q install advertools
!pip -q install pyld
!pip -q install kglab
!pip - qinstall requests
!pip -q install json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.4/286.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyld: filename=PyLD-2.0.3-py3-none-any.whl size=70847 sha256=62699825eb3f9b3f5ce6f08d9ec81b40590999b6c3a6248d1aed43ad4809587e
  Stored in directory: /root/.cache/pip/wheels/83/f1/37/b168e54c94545d0d41ddbcca7008cbd6f51fc967be7414309b
Successfully built pyld
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import advertools as adv
import json
import requests
from pyld import jsonld
from bs4 import BeautifulSoup
import urllib.request
import logging
import traceback
import kglab
import pandas as pd


In [3]:
%%time

# smurl = 'https://opencoredata.org/sitemap_0.xml'
# smurl = 'https://www.bco-dmo.org/sitemap.xml'
# smurl = 'https://opentopography.org/sitemap.xml'
# smurl = 'https://www2.earthref.org/MagIC/contributions.sitemap.xml'
# smurl = 'https://oceanscape.org/organisation-sitemap.xml'
# smurl = 'https://catalogue.cioosatlantic.ca/sitemap/sitemap-1.xml'
# smurl = 'https://obis.org/sitemap/sitemap_datasets.xml'
# smurl = 'https://infohub.eurocean.net/sitemap/vessels'
# smurl = 'https://raw.githubusercontent.com/iodepo/odis-arch/schema-dev-jm/code/notebooks/Exploration/data-pacificdatahub/sitemap.xml'
# smurl = 'https://catalogue.cioospacific.ca/sitemap/sitemap-1.xml'
# smurl = 'https://geonode.nodc.id/sitemap.xml'
# smurl = "https://lipdverse.org/sitemap.xml"
# smurl = "https://pacificdata.org/organization/sitemap.xml"
# smurl = "https://pacificdata.org/data/sitemap/cookislands-data/sitemap.xml"
# smurl = 'https://geoconnex.us/sitemap/ref/states/states__0.xml'
# smurl = "https://oceanscape.org/organisation-sitemap.xml"
# smurl = "https://catalogue.cioos.ca/sitemap/sitemap.xml"
# smurl = "https://geoconnex.us/sitemap.xml"
# smurl = "https://ioc-africa.org/sitemap.xml"
# smurl = "https://researchdata.edu.au/home/sitemap/?ds=64"
# smurl = "https://uatweb.spc.links.com.au/sitemap.xml"
# smurl = "https://uatweb.spc.links.com.au/sitemap.xml"
# smurl = "https://pacificdata.org/organization/sitemap.xml"
# smurl = "https://raw.githubusercontent.com/BeBOP-OBON/odis-interface/main/sitemap.xml"
# smurl = "https://researchdata.edu.au/home/sitemap/?ds=64"
# smurl = "https://catalogue.cioos.ca/sitemap/sitemap.xml"
# smurl = "https://api.linked-systems.uk/sitemap_pap_api.xml"
# smurl = "https://pacific-data.sprep.org/sitemap.xml"
# smurl = "https://metadata.naturalsciences.be/geonetwork/srv/api/sitemap"
# smurl = "https://emodnet.ec.europa.eu/geonetwork/srv/eng/portal.sitemap"
smurl = "https://oceanexpert.org/assets/sitemaps/sitemapIndex.xml"

iow_sitemap = adv.sitemap_to_df(smurl) # load sitemap to dataframe via advertools
iow_sitemap.info()
# iow_sitemap.head()

2023-12-07 10:31:07,818 | INFO | sitemaps.py:536 | sitemap_to_df | Getting https://www.oceanexpert.org/assets/sitemaps/sitemapTraining.xml
2023-12-07 10:31:08,522 | INFO | sitemaps.py:536 | sitemap_to_df | Getting https://www.oceanexpert.org/assets/sitemaps/sitemapEvents.xml
2023-12-07 10:31:08,522 | INFO | sitemaps.py:536 | sitemap_to_df | Getting https://www.oceanexpert.org/assets/sitemaps/sitemapInstitutions.xml
2023-12-07 10:31:09,302 | INFO | sitemaps.py:536 | sitemap_to_df | Getting https://www.oceanexpert.org/assets/sitemaps/sitemapExperts.xml


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38785 entries, 0 to 38784
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   loc                    38785 non-null  object             
 1   lastmod                38785 non-null  datetime64[ns, UTC]
 2   changefreq             38785 non-null  object             
 3   priority               38785 non-null  float64            
 4   sitemap                38785 non-null  object             
 5   etag                   38785 non-null  object             
 6   sitemap_last_modified  38785 non-null  datetime64[ns, UTC]
 7   sitemap_size_mb        38785 non-null  float64            
 8   download_date          38785 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](3), float64(2), object(4)
memory usage: 2.7+ MB
CPU times: user 1.6 s, sys: 60.3 ms, total: 1.66 s
Wall time: 3.37 s


## Analyzing the URLs

We can quickly grab the unique URLs from the sitemap column and see how many unique sitemap.xml files we are working with

We can also dive into the URL structure for the resources a bit.

In [4]:
usm = iow_sitemap.sitemap.unique()
uloc = iow_sitemap["loc"].unique()
print("{} unique sitemap XML file(s) pointing to {} unique resource(s).".format(len(usm), len(uloc)))


4 unique sitemap XML file(s) pointing to 38189 unique resource(s).


In [24]:

from urllib.parse import urlparse

invalid_urls = []

for url in uloc:
    try:
        result = urlparse(url)
        if all([result.scheme, result.netloc]):
            pass
        else:
            invalid_urls.append(url)
    except ValueError:
        print(f"{url} is an exception URL")
        invalid_urls.append(url)


df = pd.DataFrame({'Invalid URLs': invalid_urls})


AttributeError: 'float' object has no attribute 'decode'

In [25]:
df.head()

,Invalid URLs


In [26]:
from urllib.parse import urlparse

valid_urls = []


for url in uloc:
    try:
        result = urlparse(url)
        if all([result.scheme, result.netloc]):
            valid_urls.append(url)
        else:
            print(f"Invalid URL: {url}")
    except ValueError:
        print(f"Invalid URL: {url}")


AttributeError: 'float' object has no attribute 'decode'

In [27]:

# Break down all the URL into their path parts
urldf = adv.url_to_df(list(iow_sitemap['loc']))
# urldf = adv.url_to_df(list(uloc))

urldf.info()

TypeError: argument of type 'float' is not iterable

In [9]:
urldf.head()


,url,scheme,netloc,path,query,fragment,dir_1,dir_2,dir_3,last_dir
0,https://www.ltreb-reservoirs.org/vera4cast//ca...,https,www.ltreb-reservoirs.org,/vera4cast//catalog.html,NaN,NaN,vera4cast,NaN,catalog.html,catalog.html
1,https://www.ltreb-reservoirs.org/vera4cast//in...,https,www.ltreb-reservoirs.org,/vera4cast//instructions.html,NaN,NaN,vera4cast,NaN,instructions.html,instructions.html
2,https://www.ltreb-reservoirs.org/vera4cast//pe...,https,www.ltreb-reservoirs.org,/vera4cast//performance.html,NaN,NaN,vera4cast,NaN,performance.html,performance.html
3,https://www.ltreb-reservoirs.org/vera4cast//in...,https,www.ltreb-reservoirs.org,/vera4cast//index.html,NaN,NaN,vera4cast,NaN,index.html,index.html
4,https://radiantearth.github.io/stac-browser/#/...,https,radiantearth.github.io,/stac-browser/,NaN,/external/raw.githubusercontent.com/LTREB-rese...,stac-browser,NaN,NaN,stac-browser


## Sample and test sitemap entries

In [10]:
# sample the previously generated url data frame
sample_size = 5
# sample_df = urldf.groupby("dir_1").sample(n=sample_size, random_state=1, replace=True)
sample_df = urldf.sample(n=sample_size, random_state=1, replace=True)

In [11]:
sample_df.head(5)

,url,scheme,netloc,path,query,fragment,dir_1,dir_2,dir_3,last_dir
37,https://radiantearth.github.io/stac-browser/#/...,https,radiantearth.github.io,/stac-browser/,NaN,/external/raw.githubusercontent.com/LTREB-rese...,stac-browser,NaN,NaN,stac-browser
12,https://radiantearth.github.io/stac-browser/#/...,https,radiantearth.github.io,/stac-browser/,NaN,/external/raw.githubusercontent.com/LTREB-rese...,stac-browser,NaN,NaN,stac-browser
72,https://radiantearth.github.io/stac-browser/#/...,https,radiantearth.github.io,/stac-browser/,NaN,/external/raw.githubusercontent.com/LTREB-rese...,stac-browser,NaN,NaN,stac-browser
9,https://radiantearth.github.io/stac-browser/#/...,https,radiantearth.github.io,/stac-browser/,NaN,/external/raw.githubusercontent.com/LTREB-rese...,stac-browser,NaN,NaN,stac-browser
75,https://radiantearth.github.io/stac-browser/#/...,https,radiantearth.github.io,/stac-browser/,NaN,/external/raw.githubusercontent.com/LTREB-rese...,stac-browser,NaN,NaN,stac-browser


### See if the URLs resolve

In [12]:
import urllib.request
import requests

ul = sample_df["url"]

for item in ul:
    # user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    # headers={'User-Agent':user_agent,}

    headers= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                           'AppleWebKit/537.11 (KHTML, like Gecko) '
                           'Chrome/23.0.1271.64 Safari/537.11',
             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
             'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
             'Accept-Encoding': 'none',
             'Accept-Language': 'en-US,en;q=0.8',
             'Connection': 'keep-alive'}

    try:
        # x = requests.get(item)
        # code = x.status_code
        request=urllib.request.Request(url=item, headers=headers) #The assembled request
        with urllib.request.urlopen(request) as response:
            info = response.info()
            dtype = info.get_content_type()    # -> text/html
         # headers = x.headers()
        # print("URL: {} \ninfo : {}\n --".format(item, info))
        print("URL: {} ".format(item))
    except Exception as e:
        # code = x.status_code
        # dtype = info.get_content_type()

        print("Exception on: {} \nerrors : {}\n --".format(item, str(e)))


URL: https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/noaa_forecasts/Stage1-stats/collection.json 
URL: https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/forecasts/Physical/Daily_Water_temperature/collection.json 
URL: https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/summaries/Biological/Daily_Chlorophyll-a/collection.json 
URL: https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/forecasts/Chemical/Daily_oxygen_concentration/collection.json 
URL: https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/summaries/Chemical/collection.json 


### See if they have JSON-LD (static check only, no dynamically loaded JSON-LD yet)

In [13]:
ul = sample_df["url"]

headers= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                        'AppleWebKit/537.11 (KHTML, like Gecko) '
                        'Chrome/23.0.1271.64 Safari/537.11',
          'Accept': 'application/ld+json,text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
          'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
          'Accept-Encoding': 'none',
          'Accept-Language': 'en-US,en;q=0.8',
          'Connection': 'keep-alive'}

for item in ul:
    print(item)
    request=urllib.request.Request(url=item, headers=headers)
    # p = urllib.request.urlopen(request).read()

    html = urllib.request.urlopen(request).read()
    soup = BeautifulSoup(html, "html.parser")
    p = soup.find('script', {'type':'application/ld+json'})
    try:
        # print("JSON byte size: {} ".format(len(p)))
        print(p)
        print("JSON byte size: {} ".format(len(p.contents[0])))
    except Exception as e:
        logging.error(traceback.format_exc())

2023-12-03 18:54:40,306 | ERROR | <ipython-input-13-d236e4a390d7>:25 | <module> | Traceback (most recent call last):
  File "<ipython-input-13-d236e4a390d7>", line 23, in <module>
    print("JSON byte size: {} ".format(len(p.contents[0])))
AttributeError: 'NoneType' object has no attribute 'contents'

2023-12-03 18:54:40,356 | ERROR | <ipython-input-13-d236e4a390d7>:25 | <module> | Traceback (most recent call last):
  File "<ipython-input-13-d236e4a390d7>", line 23, in <module>
    print("JSON byte size: {} ".format(len(p.contents[0])))
AttributeError: 'NoneType' object has no attribute 'contents'

2023-12-03 18:54:40,405 | ERROR | <ipython-input-13-d236e4a390d7>:25 | <module> | Traceback (most recent call last):
  File "<ipython-input-13-d236e4a390d7>", line 23, in <module>
    print("JSON byte size: {} ".format(len(p.contents[0])))
AttributeError: 'NoneType' object has no attribute 'contents'

2023-12-03 18:54:40,454 | ERROR | <ipython-input-13-d236e4a390d7>:25 | <module> | Traceback

https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/noaa_forecasts/Stage1-stats/collection.json
None
https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/forecasts/Physical/Daily_Water_temperature/collection.json
None
https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/summaries/Biological/Daily_Chlorophyll-a/collection.json
None
https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/forecasts/Chemical/Daily_oxygen_concentration/collection.json
None
https://radiantearth.github.io/stac-browser/#/external/raw.githubusercontent.com/LTREB-reservoirs/vera4cast/main/catalog/summaries/Chemical/collection.json


2023-12-03 18:54:40,510 | ERROR | <ipython-input-13-d236e4a390d7>:25 | <module> | Traceback (most recent call last):
  File "<ipython-input-13-d236e4a390d7>", line 23, in <module>
    print("JSON byte size: {} ".format(len(p.contents[0])))
AttributeError: 'NoneType' object has no attribute 'contents'



None


### Check JSON-LD structure (static check only, no dynamically loaded JSON-LD yet)

In [14]:
ul = sample_df["url"]

myframe =  {
    "@context":{"@vocab": "http://schema.org/"},
    "@type": "Dataset",
}

context =  { "@vocab": "http://schema.org/" }

headers= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                        'AppleWebKit/537.11 (KHTML, like Gecko) '
                        'Chrome/23.0.1271.64 Safari/537.11',
          'Accept': 'application/ld+json,text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
          'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
          'Accept-Encoding': 'none',
          'Accept-Language': 'en-US,en;q=0.8',
          'Connection': 'keep-alive'}

for item in ul:
    request=urllib.request.Request(url=item, headers=headers)
    # p = urllib.request.urlopen(request).read()
    html = urllib.request.urlopen(request).read()
    soup = BeautifulSoup(html, "html.parser")
    p = soup.find('script', {'type':'application/ld+json'})
    try:
        jld = json.loads(p.contents[0])
        # jld = json.loads(p)

        # print(str(jld))
        # compacted = jsonld.compact(str(jld), context)
        # print(len(json.dumps(compacted, indent=2)))
    except Exception as e:
        print("Exception")
        logging.error(traceback.format_exc())

2023-12-03 18:54:42,270 | ERROR | <ipython-input-14-9d11dcc3fcb5>:34 | <module> | Traceback (most recent call last):
  File "<ipython-input-14-9d11dcc3fcb5>", line 26, in <module>
    jld = json.loads(p.contents[0])
AttributeError: 'NoneType' object has no attribute 'contents'

2023-12-03 18:54:42,319 | ERROR | <ipython-input-14-9d11dcc3fcb5>:34 | <module> | Traceback (most recent call last):
  File "<ipython-input-14-9d11dcc3fcb5>", line 26, in <module>
    jld = json.loads(p.contents[0])
AttributeError: 'NoneType' object has no attribute 'contents'

2023-12-03 18:54:42,368 | ERROR | <ipython-input-14-9d11dcc3fcb5>:34 | <module> | Traceback (most recent call last):
  File "<ipython-input-14-9d11dcc3fcb5>", line 26, in <module>
    jld = json.loads(p.contents[0])
AttributeError: 'NoneType' object has no attribute 'contents'

2023-12-03 18:54:42,427 | ERROR | <ipython-input-14-9d11dcc3fcb5>:34 | <module> | Traceback (most recent call last):
  File "<ipython-input-14-9d11dcc3fcb5>", line

Exception
Exception
Exception
Exception


2023-12-03 18:54:42,475 | ERROR | <ipython-input-14-9d11dcc3fcb5>:34 | <module> | Traceback (most recent call last):
  File "<ipython-input-14-9d11dcc3fcb5>", line 26, in <module>
    jld = json.loads(p.contents[0])
AttributeError: 'NoneType' object has no attribute 'contents'



Exception


## Load to Graph"

Looad a sample set of triples into RDF lib and run a sample SPARQL query on them.

### Note
This is the same loop as above but now we will load into a KG graph

In [ ]:
ul = sample_df["url"]

# Test loading into a graph
namespaces = {
    "schema":  "http://schema.org/",
    "schemaold":  "http://schema.org/",
    "shacl":   "http://www.w3.org/ns/shacl#" ,
}

kg = kglab.KnowledgeGraph(
    name = "Schema.org shacl eval datagraph",
    base_uri = "https://example.org/id/",
    namespaces = namespaces,
)

for item in ul:
    html = urllib.request.urlopen(item).read()
    soup = BeautifulSoup(html, "html.parser")
    p = soup.find('script', {'type':'application/ld+json'})
    try:
        print("JSON byte size: {} ".format(len(p.contents[0])))
        kg.load_rdf_text(data=p.contents[0], format="json-ld")
        print(p.contents[0])
    except Exception as e:
        logging.error(traceback.format_exc())

HTTPError: HTTP Error 403: Forbidden

In [ ]:
sparql = """
PREFIX schema: <http://schema.org/>
SELECT ?s ?name ?description ?type
  WHERE {
    ?s a ?type .
    ?s schema:name ?name .
    ?s schema:description ?description.
  }
"""

#  schema:Dataset
# ?s schema:name ?name .
#     ?s schema:description ?description.

df = kg.query_as_df(sparql).to_pandas()

df.head()

AttributeError: 'DataFrame' object has no attribute 'to_pandas'